# Notebook 7: YOLO Pascal VOC Data

En este notebook vamos a ver como se entrena la red YOLO desde cero utilizando un dataset llamado Pascal VOC. 

### 1- Recolección del dataset
Para entrenar YOLO nos hace falta un conjunto de imágenes que formen el dataset. En este caso el dataset está formado por las imágenes de la base de datos de [Pascal VOC](http://host.robots.ox.ac.uk/pascal/VOC/) desde 2007 hasta 2012. Este dataset fue diseñado con el fin de crear y evaluar algoritmos para la clasificación de imágenes, la detección de objetos y la segmentación. Además del dataset, el proyecto Pascal VOC proporciona herramientas que permiten, por un lado, acceder a los conjuntos de datos y anotaciones; y, por otro, evaluar y comparar diferentes métodos. Este dataset cuenta con un total de 54000 imágenes de las que nosotros sólo vamos a usar 21.503, que contienen objetos de 20 clases distintas divididas en cuatro grupos: persona, animales, vehículos e interior. Estos cuatro grupos se dividen en las siguientes clases: 
* Persona: persona.
* Animal: pájaro, gato, vaca, perro, caballo y oveja.
* Vehículo: avión, bicicleta, barco, autobús, coche, moto y tren.
* Interior: botella, silla, mesa de comedor, planta en maceta, sofá y tv / monitor.

Para obtener estos datos hay que ejecutar los siguientes comandos que descargan y descomprimen el dataset.

In [0]:
!wget https://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar
!wget https://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar
!wget https://pjreddie.com/media/files/VOCtest_06-Nov-2007.tar
!tar xf VOCtrainval_11-May-2012.tar
!tar xf VOCtrainval_06-Nov-2007.tar
!tar xf VOCtest_06-Nov-2007.tar

Estas instrucciones nos habrán generado un directorio VOCdevkit donde se habrán descargado todas las imágenes del dataset Pascal VOC. Interesa sobre todo remarcar que en ese directorio hay 5 carpetas de imágenes: 2007_train, 2007_val, 2007_test, 2012_train y 2012_val. Y además también hay una carpeta de anotaciones con 5 subcarpetas, una para cada carpeta de imágenes. 

### 2- Data augmentation
En este caso al tener un dataset de 21.503 imágenes, no ha sido necesario aumentarlo aún más.

### 3- Anotación del dataset
Ahora necesitamos que todas las imágenes están anotadas, es decir,  YOLO necesita que cada imagen tenga un archivo .txt con una  línea por cada objeto que nos interese detectar. El dataset Pascal VOC ya viene anotado pero no utiliza el mismo formato que YOLO. En concreto Pascal VOC utiliza un formato propio en XML y que se usa habitualmente para anotar objetos. Por lo tanto es necesario transformar las anotaciones del formato Pascal VOC a anotaciones formato YOLO, para lo cual hay que utilizar el script  *voc_label.py* proporcionado por la librería YOLO.

In [0]:
!wget https://pjreddie.com/media/files/voc_label.py
!python voc_label.py

Este comando nos genera para cada una de las carpetas de imágenes, un fichero .txt con la lista de imágenes de dicha carpeta, los ficheros generados son: 2007_test.txt, 2007_train.txt, 2007_val.txt, 2012_train.txt y 2012_val.txt, ver siguiente Figura. Una carpeta labels dentro del directorio VOCdevkit que contendrá las anotaciones para YOLO. Y para cada imagen del dataset de YOLO se ha creado en el directorio labels un archivo .txt con una línea para cada objeto a detectar en dicha imagen.

<img src="imágenes/anotacionesVOC.png" style="width:400px;">



### 4- Dataset split
Este paso es un poco distinto a lo que se hace habitualmente. Normalmente se tiene un conjunto de imágenes y el usuario es el encargado de partirlas en dos conjuntos, uno de entrenamiento y otro de test, en cambio en los desafíos o competiciones como Pascal VOC se proporcionan explicitamente un conjunto de entrenamiento y un conjunto de test para que todos los participantes del concurso evalúen contra el mismo dataset.

Para este caso, el conjunto de entrenamiento será todas las imágenes salvo el conjunto de test del 2007. Como YOLO necesita un archivo .txt con todas las imágenes que queremos que entrene, debemos ejecutar la siguiente instrucción que nos combina las listas de imágenes de los conjuntos de entrenamiento y validación de 2007 y 2012. 

In [0]:
!cat 2007_train.txt 2007_val.txt 2012_*.txt > train.txt

Ahora tenemos una lista con todas las imágenes con las que vamos a entrenar en el fichero train.txt, y otra lista con las imágenes  para evaluar en el fichero 2007_test.txt. Se han usado 16551 imágenes para el entrenamiento y 4952 para realizar el test.


### 5- Entrenamiento
El primer paso antes de empezar a entrenar será cambiar el archivo de configuración cfg/voc.data para que apunte a las imágenes que hemos descargado. 
<img src="imágenes/voc.png" style="width:500px;">

A parte de este fichero también es necesario que modifiquemos el fichero cfg/yolov3-voc.cfg, indicando que vamos a realizar el entrenamiento:
<img src="imágenes/train.png" style="width:250px;">

Hay dos maneras de entrenar la red YOLO: o bien usando unos pesos pre-entrenados o partiendo de cero. En este caso se van a usar unos pesos previamente pre-entrenados llamados *darknet53.conv.74*, estos pesos que usamos provienen del modelo darknet53. El primer paso consistirá en descargarlos.

In [0]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

A continuación comenzamos con el entrenamiento:

In [0]:
!./darknet detector train cfg/voc.data cfg/yolov3-voc.cfg darknet53.conv.74

Falta la epoca, tiempo que le cuesta

### 6- Evaluación
Una vez finalizado el entrenamiento, y antes de comprobar que nos funciona en imágenes reales, vamos a evaluar el modelo construido en el conjunto de test. Como entrenar un modelo para Pascal VOC lleva mucho tiempo vamos a coger los pesos ya entrenados que son proporcionados en la página de [Alexey](https://github.com/AlexeyAB/darknet). Para poder ejecutar este test es necesario descargarse una librería [darknet](https://github.com/AlexeyAB/darknet) desde otro sitio que tiene un script que permite ejecutar la siguiente instrucción. Además también hará falta configurar el archivo voc.data, donde indiquemos las rutas de las listas de imágenes. Una vez realizado este paso, ejecutamos la siguiente instrucción.

In [0]:
./darknet detector map cfg/voc.data cfg/yolo-voc.cfg yolo-voc.weights

Al acabar, el comando anterior nos mostrará el valor de AP para cada clase del dataset, y el valor global de map, lo que nos permite clasificarlo como aceptable o no.

<img src="imágenes/map.png" style="width:500px;">

### 7- Predicción
Por último probamos nuestro modelo con imágenes que no se encontraran en el conjunto inicial. Ejecutamos para ello la siguiente instrucción con una imagen.

In [0]:
./darknet detector test cfg/voc.data cfg/yolo-voc.cfg yolo-voc.weights data/dog.jpg 

Vemos que ha detectado que en esta imagen hay un perro, un coche y una bicicleta:

<img src="imágenes/predictions1.png" style="width:500px;">

Y en la imagen resultado lo vemos:

<img src="imágenes/predictions.png" style="width:500px;">